Baixar a base de dados no drive, link: https://drive.google.com/drive/folders/1aQ4VGvA9XRgvb_iBMtx1CTdype8LARRt?usp=drive_link


Alunos: Rafael Pascoali Czerniej, Gabriel Yudi, Nathan Oliboni, André Henrique

In [4]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib #baixamos as bibliotecas necessárias
!pip list
!pip install tensorboard
!pip install -q -U keras-tuner

EPOCAS = 10 #!!!!!!!
import cv2
import tensorflow as tf
import os
import imghdr
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential # API sequencial, boa para quando temos apenas um valor de entrada
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout # camadas do keras
from tensorflow.keras.callbacks import TensorBoard
import keras_tuner as kt
from kerastuner import Hyperband

Defaulting to user installation because normal site-packages is not writeable
INFO: pip is looking at multiple versions of tensorflow-gpu to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
ERROR: Requested tensorflow-gpu from https://files.pythonhosted.org/packages/8a/45/fa31ced1db38f9424f262dfbf35747fe5378b5c808cecb373c8cb8e515d3/tensorflow-gpu-2.12.0.tar.gz has invalid metadata: Expected end or semicolon (after name and no valid version specifier)
    python_version>"3.7"
                  ^
Package                      Version
---------------------------- ----------------
absl-py                      2.1.0
anyio                        4.0.0
appdirs                      1.4.4
apt-clone                    0.2.1
apturl                       0.5.2
argcomplete                  1.8.1
argon2-cffi                  23.1.0
argon2-cffi-bindings         21.2.0
ar

2024-10-27 21:32:54.210480: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-27 21:32:54.284121: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-27 21:32:54.357724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-27 21:32:54.424898: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-27 21:32:54.442894: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-27 21:32:54.579066: I tensorflow/core/platform/cpu_feature_gu

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('./content/data/') #Tensorflow coleta as imagens do diretório
class_names = data.class_names
print(class_names) #printa as classes encontradas pelo Keras
data_iterator = data.as_numpy_iterator()  #organiza os dados para um iterador no numpy

batch = data_iterator.next() #batch é a coleção das imagens da base em formato matricial

fig, ax = plt.subplots(ncols=4, figsize=(300,300))
for idx, img in enumerate(batch[0][:4]): #codigo para mostrar exemplares dos datasets
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

Found 9469 files belonging to 2 classes.
['Boas', 'Podres']


2024-10-27 21:33:15.682012: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25165824 exceeds 10% of free system memory.
2024-10-27 21:33:15.757733: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25165824 exceeds 10% of free system memory.
2024-10-27 21:33:15.826281: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25165824 exceeds 10% of free system memory.


In [ ]:
data = data.map(lambda x,y: (x/255, y))
data.as_numpy_iterator().next()

train_size = int(len(data)*.7) # treino sera 70% dos dados
val_size = int(len(data)*.2) # validação 20%
test_size = int(len(data)*.1) # teste será 10%

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

Usamos uma função construtora para gerar o modelo, o parametro hp é passado automaticamente pelo tuner do keras e não precisamos nos preocupar com ele agora

In [ ]:
def model_builder(hp):
  model = keras.Sequential() # cria o modelo sequencial
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  hp_units = hp.Int('Units', min_value=32, max_value=512,step = 32)
#camadas adicionadas sequencialmente
# primeira camada convolucional tem 16 filtros de tamanho 3x3
# Relu está substituindo valores nulos.
  model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
# segunda camada - escolhe o maior valor apos a função relu ativar (2 x 2)
  model.add(MaxPooling2D())
# terceira camada, saida tem apenas um valor, use 32 filtros
  model.add(Conv2D(32, (3,3), 1, activation='relu'))
# quarta camada
  model.add(MaxPooling2D())
# quinta camada
  model.add(Conv2D(16, (3,3), 1, activation='relu'))
# sexta camada
  model.add(MaxPooling2D())
# usamos uma função flatten para transformar os dados em um vetor unidimencional
  model.add(Flatten())
# 2 camadas densas de convolução, quantidade de nuronios da primeira camada definida na procura por hiperparametros
  model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))   # Primeira camada densa, com a quantidade de nuronios encontrada na busca


# Última camada sigmoide, para classificação
  model.add(Dense(1, activation='sigmoid')) #a ultima camada, que nos dará o resultado de se a fruta está podre ou boa.
# compilamos e retornamos o modelo, com a taxa de aprendizado definida na busca por hiperparâmetros
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),loss=keras.losses.BinaryCrossentropy(from_logits=False),metrics=['accuracy'])
  return model

Utilizamos o Hyperband, o avaliador de hiperparamtros do Keras, pois funciona melhor com redes neurais do Keras

In [ ]:
#instanciar hypertuner para o processo de busca de hiperparametros
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy', #aumentar a precisão da predição
                     max_epochs=EPOCAS,
                     factor=3, #fator de regressão, valor padrão é 3
                     directory='./content/',
                     project_name='RedeNeuralTrabalho',
                     overwrite=True)

stop_early = tf.keras.callbacks.EarlyStoprping(monitor='val_loss', patience=5)
#definir um critério de parada do hypertuner, se em 5 iterações o val_loss não melhorou, ele para

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
stop_early = tf.keras.callbacks.EarlyStoprping(monitor='val_loss', patience=5)
#definir um critério de parada do hypertuner, se em 5 iterações o val_loss não melhorou, ele para

após definirmos a estrutura da RNA e do Hypertuner, podemos iniciar a busca de hiperparametros

In [ ]:
#iniciar busca, mesmos parametros que model.fit
tuner.search(train, epochs=EPOCAS, validation_data=val, callbacks=[stop_early])
#essa execução demora um bocado


Trial 11 Complete [00h 19m 07s]
val_accuracy: 0.9152542352676392

Best val_accuracy So Far: 0.9427965879440308
Total elapsed time: 03h 26m 04s

Search: Running Trial #12

Value             |Best Value So Far |Hyperparameter
320               |96                |Units
0.001             |0.001             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
 82/207 ━━━━━━━━━━━━━━━━━━━━ 4:31 2s/step - accuracy: 0.6526 - loss: 0.6350

KeyboardInterrupt: 

best_hps guarda os melhores hiperparametros encontratos após a busca, metodos get necessários para acessar os dados

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""Busca completa, o numero de units para a primeira camada densa é {best_hps.get('Units')} e a melhor taxa de aprendizado é {best_hps.get('learning_rate')}.""")


Busca completa, o numero de units para a primeira camada densa é 96 e a melhor taxa de aprendizado é 0.001.


agora que sabemos os melhores hiperparametros, podemos construir a RNA com os valores ideais

In [ ]:
model = tuner.hypermodel.build(best_hps)


In [ ]:
# Definir o callback do tensorboard
log_dir = "logs"  # Diretório para guardar as logs
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# usamos model.fit para treinar o modelo
hist = model.fit(train, epochs=EPOCAS, validation_data=val, callbacks=[tensorboard_callback])

hist.history

Epoch 1/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 572s 3s/step - accuracy: 0.7209 - loss: 0.4955 - val_accuracy: 0.9115 - val_loss: 0.2185
Epoch 2/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 543s 3s/step - accuracy: 0.9086 - loss: 0.2295 - val_accuracy: 0.9195 - val_loss: 0.2027
Epoch 3/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 578s 3s/step - accuracy: 0.9444 - loss: 0.1479 - val_accuracy: 0.9428 - val_loss: 0.1479
Epoch 4/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 573s 3s/step - accuracy: 0.9620 - loss: 0.1101 - val_accuracy: 0.9513 - val_loss: 0.1267
Epoch 5/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 522s 3s/step - accuracy: 0.9687 - loss: 0.0831 - val_accuracy: 0.9492 - val_loss: 0.1408
Epoch 6/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 576s 3s/step - accuracy: 0.9788 - loss: 0.0626 - val_accuracy: 0.9613 - val_loss: 0.1070
Epoch 7/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 571s 3s/step - accuracy: 0.9863 - loss: 0.0399 - val_accuracy: 0.9730 - val_loss: 0.0721
Epoch 8/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 571s 3s/step - accuracy: 0.9887 - loss: 0.0352 - val_accu

{'accuracy': [0.8087258338928223,
  0.9211956262588501,
  0.9486715197563171,
  0.9645229578018188,
  0.9648248553276062,
  0.98097825050354,
  0.9842995405197144,
  0.9880736470222473,
  0.9930555820465088,
  0.9910929799079895],
 'loss': [0.389121949672699,
  0.2064213752746582,
  0.1379280388355255,
  0.10084638744592667,
  0.09393128752708435,
  0.058104053139686584,
  0.04145026579499245,
  0.033132050186395645,
  0.022014625370502472,
  0.023372098803520203],
 'val_accuracy': [0.9115465879440308,
  0.9194915294647217,
  0.9427965879440308,
  0.9512711763381958,
  0.9491525292396545,
  0.961334764957428,
  0.9729872941970825,
  0.9735169410705566,
  0.9745762944221497,
  0.9634534120559692],
 'val_loss': [0.21846047043800354,
  0.20266717672348022,
  0.14786866307258606,
  0.12671206891536713,
  0.14081673324108124,
  0.10703326016664505,
  0.07208099216222763,
  0.08801285177469254,
  0.08890363574028015,
  0.13198229670524597]}

abaixo, analisamos a precisão e o loss do modelo

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='silver', label='loss')
plt.plot(hist.history['val_loss'], color='tan', label='val_loss')
fig.suptitle('Loss', fontsize=10)
plt.legend(loc="upper left")
plt.show()


fig = plt.figure()
plt.plot(hist.history['accuracy'], color='silver', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='tan', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=10)
plt.legend(loc="upper left")
plt.show()

encontramos a melhor época no histórico de treinamento

In [ ]:
val_acc_por_epoca = hist.history['val_accuracy']
melhor_epoca = val_acc_por_epoca.index(max(val_acc_por_epoca)) + 1
print('Melhor época: ', melhor_epoca)


Melhor época:  9


agora, treinamos o modelo com a melhor época e melhores hiperparamtros


In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(train, epochs=best_epoch, validation_data=val)


Epoch 1/9


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


207/207 ━━━━━━━━━━━━━━━━━━━━ 525s 3s/step - accuracy: 0.7203 - loss: 0.5140 - val_accuracy: 0.8994 - val_loss: 0.2463
Epoch 2/9
207/207 ━━━━━━━━━━━━━━━━━━━━ 571s 3s/step - accuracy: 0.9041 - loss: 0.2409 - val_accuracy: 0.9237 - val_loss: 0.1905
Epoch 3/9
207/207 ━━━━━━━━━━━━━━━━━━━━ 522s 3s/step - accuracy: 0.9156 - loss: 0.2006 - val_accuracy: 0.9311 - val_loss: 0.1762
Epoch 4/9
207/207 ━━━━━━━━━━━━━━━━━━━━ 569s 3s/step - accuracy: 0.9370 - loss: 0.1541 - val_accuracy: 0.9587 - val_loss: 0.1193
Epoch 5/9
207/207 ━━━━━━━━━━━━━━━━━━━━ 551s 3s/step - accuracy: 0.9490 - loss: 0.1277 - val_accuracy: 0.9518 - val_loss: 0.1331
Epoch 6/9
207/207 ━━━━━━━━━━━━━━━━━━━━ 519s 3s/step - accuracy: 0.9654 - loss: 0.0944 - val_accuracy: 0.9640 - val_loss: 0.0979
Epoch 7/9
207/207 ━━━━━━━━━━━━━━━━━━━━ 574s 3s/step - accuracy: 0.9655 - loss: 0.0856 - val_accuracy: 0.9550 - val_loss: 0.1401
Epoch 8/9
207/207 ━━━━━━━━━━━━━━━━━━━━ 530s 3s/step - accuracy: 0.9799 - loss: 0.0565 - val_accuracy: 0.9656 - val

Abaixo temos testes automatizados com datasets separados que criamos manualmente

In [ ]:

arquivos = [f for f in os.listdir('./content/data/test_boas') if f.endswith('.jpg')]
certos = 0
errados = 0
fotos_errados = []
for arquivo in arquivos:
  caminhoImagem = os.path.join('./content/data/test_boas', arquivo)
  img = cv2.imread(caminhoImagem) #transforma a imagem em uma matriz cv2
  img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #converto para RGB (CV2 usa GBR para representar cores)
  resize = tf.image.resize(img, (256,256)) #mudo o tamanho para 256x256
  np.expand_dims(resize,0)
  test_prediction = hypermodel.predict(np.expand_dims(resize/255, 0)) #faz a predeição com os valores x normalizados
  test_prediction
  if test_prediction < 0.5: # se a predição é menor que 0.5, está certo
    certos = certos + 1
  else:
    errados = errados + 1
    fotos_errados.append(arquivo)
print('certos:', certos)
print('errados', errados)
print('onde errou:', fotos_errados)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━

In [ ]:
arquivos = [f for f in os.listdir('./content/data/test_podres') if f.endswith('.jpg')]
certos = 0
errados = 0
fotos_errados = []
for arquivo in arquivos:
  caminhoImagem = os.path.join('./content/data/test_podres', arquivo)
  img = cv2.imread(caminhoImagem)
  img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  resize = tf.image.resize(img, (256,256))
  np.expand_dims(resize,0)
  test_prediction = hypermodel.predict(np.expand_dims(resize/255, 0))
  8.380818e-05
  test_prediction
  if test_prediction > 0.5:
    certos = certos + 1
  else:
    errados = errados + 1
    fotos_errados.append(arquivo)
print('certos:', certos)
print('errados', errados)
print('onde errou:', fotos_errados)c:\Users\Nathan Oliboni\Desktop\Rede Neural\Site\index.html

NameError: name 'os' is not defined

Após verificar o funcionamento do modelo, podemos salva-lo e carrega-lo:

In [ ]:
# Salvar o modelo
hypermodel.save('hypermodel.keras')
#carregar o modelo
hypermodel2 = tf.keras.models.load_model('./content/data/test_boas')

NameError: name 'hypermodel' is not defined

In [ ]:
arquivos = [f for f in os.listdir('drive/MyDrive/content/datatest/test_podres') if f.endswith('.jpg')]
certos = 0
errados = 0
for arquivo in arquivos:
  caminhoImagem = os.path.join('drive/MyDrive/content/datatest/test_podres', arquivo)
  img = cv2.imread(caminhoImagem)
  img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  resize = tf.image.resize(img, (256,256))
  np.expand_dims(resize,0)
  test_prediction = hypermodel2.predict(np.expand_dims(resize/255, 0))
  8.380818e-05
  test_prediction
  if test_prediction > 0.5:
    certos = certos + 1
  else:
    errados = errados + 1
print('certos:', certos)
print('errados', errados)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━

In [ ]:
arquivos = [f for f in os.listdir('drive/MyDrive/content/datatest/test_boas') if f.endswith('.jpg')]
certos = 0
errados = 0
for arquivo in arquivos:
  caminhoImagem = os.path.join('drive/MyDrive/content/datatest/test_boas', arquivo)
  img = cv2.imread(caminhoImagem)
  img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  resize = tf.image.resize(img, (256,256))
  np.expand_dims(resize,0)
  test_prediction = hypermodel2.predict(np.expand_dims(resize/255, 0))
  8.380818e-05
  test_prediction
  if test_prediction < 0.5:
    certos = certos + 1
  else:
    errados = errados + 1
print('certos:', certos)
print('errados', errados)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━

Com o funcionamento da RNA verificado, e vendo que suas funções de carregar e salvar funcionam corretamente, podemos obter a matriz de confusão

In [ ]:

# Obtenha os rótulos verdadeiros e as previsões para o conjunto de testes
verdadeiros = []
predicao = []

for imagens, labels in test:
    preds = model.predict(imagens)
    verdadeiros.extend(labels.numpy())
    predicao.extend((preds > 0.5).astype(int))

# Gere a matriz de confusão
mc = confusion_matrix(verdadeiros, predicao)
plt.figure(figsize=(10, 8))
sns.heatmap(mc, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predição")
plt.ylabel("Verdadeiro")
plt.show()

NameError: name 'model' is not defined